# Geração de Medidas Coerência Tópica do conjunto de dados CohQuAD In en

Este notebook, realiza testes de medidas de (in)coerência entre pares de documentos do conjunto de dados utilizando tópico.

A medida de um documento(D) é realizada pela coerência tópica do documento. Todo o documento é submetido e as medidas recuperadas. Cada documento é analisado pelas medidas **Cc_uci**, **Cc_npmi**, **Cc_v** e **Cu_mass**. Estas medidas de coerência utilizando a coerência tópica utilizando as palavras dos documentos.

As seguintes medidas foram calculadas entre os embeddings das sentenças **Si** e **Sj**:

A medida **c_uci** é baseada em uma janela deslizante e nas informações mútuas pontuais (PMI) de todos os pares de palavras das principais palavras fornecidas

**c_npmi** é uma versão aprimorada da coerência C_uci usando a informação mútua pontual normalizada (NPMI)

**c_v** é baseada em uma janela deslizante, segmentação de um conjunto das principais palavras e uma medida de confirmação indireta que usa informações mútuas pontuais normalizadas (NPMI) e a semelhança de cosseno

**c_umass** é baseado em contagens de coocorrência de documentos, uma segmentação de um precedente e uma probabilidade condicional logarítmica como medida de confirmação

Utiliza os arquivos para gerar as medidas:
- `original.zip`
- `originalpos.zip`
- `perturbado_pX_kY.zip`
- `perturbadopos_pX_kY.zip`
- `corpus_especifico.zip`

Nos nomes dos arquivos, `X` é o número de documentos perturbados e `Y` o valor de top `K` predições.


----------------------------

**Link biblioteca Transformers:**
https://github.com/huggingface/transformers


**Artigo original BERT:**
https://arxiv.org/pdf/1506.06724.pdf

# 1 Preparação do ambiente
Preparação do ambiente para execução do exemplo.

## 1.1 Tempo inicial de processamento

In [ ]:
# Import das bibliotecas
import time
import datetime

#marca o tempo de início do processamento.
inicio_processamento = time.time()

## 1.2 Funções e classes auxiliares

Verifica se existe o diretório cohebert no diretório corrente.   


In [ ]:
# Import das bibliotecas.
import os # Biblioteca para manipular arquivos

# ============================
def verificaDiretorioCoheBERT():
    """
      Verifica se existe o diretório cohebert no diretório corrente.
    """

    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_COHEBERT):
        # Cria o diretório
        os.makedirs(DIRETORIO_COHEBERT)
        logging.info("Diretório Cohebert criado: {}".format(DIRETORIO_COHEBERT))

    return DIRETORIO_COHEBERT

Realiza o download e um arquivo

In [ ]:
# Import das bibliotecas.
import requests # Biblioteca de download
from tqdm.notebook import tqdm as tqdm_notebook # Biblioteca para barra de progresso
import os # Biblioteca para manipular arquivos

def downloadArquivo(url_arquivo, nome_arquivo_destino):
    """
      Realiza o download de um arquivo de uma url em salva em nome_arquivo_destino.

      Parâmetros:
        `url_arquivo` - URL do arquivo a ser feito download.
        `nome_arquivo_destino` - Nome do arquivo a ser salvo.
    """

    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()

    # Realiza o download de um arquivo em uma url
    data = requests.get(url_arquivo, stream=True)

    # Verifica se o arquivo existe
    if data.status_code != 200:
        logging.info("Exceção ao tentar realizar download {}. Response {}.".format(url_arquivo, data.status_code))
        data.raise_for_status()
        return

    # Recupera o nome do arquivo a ser realizado o download
    nome_arquivo = nome_arquivo_destino.split("/")[-1]

    # Define o nome e caminho do arquivo temporário
    nome_arquivo_temporario = DIRETORIO_COHEBERT + "/" + nome_arquivo + "_part"

    logging.info("Download do arquivo: {}.".format(nome_arquivo_destino))

    # Baixa o arquivo
    with open(nome_arquivo_temporario, "wb") as arquivo_binario:
        tamanho_conteudo = data.headers.get("Content-Length")
        total = int(tamanho_conteudo) if tamanho_conteudo is not None else None
        # Barra de progresso de download
        progresso_bar = tqdm_notebook(unit="B", total=total, unit_scale=True)
        # Atualiza a barra de progresso
        for chunk in data.iter_content(chunk_size=1024):
            if chunk:
                progresso_bar.update(len(chunk))
                arquivo_binario.write(chunk)

    # Renomeia o arquivo temporário para o arquivo definitivo
    os.rename(nome_arquivo_temporario, nome_arquivo_destino)

    # Fecha a barra de progresso.
    progresso_bar.close()

Remove tags de um documento

In [ ]:
def remove_tags(documento):
    """
      Remove tags de um documento
    """

    import re

    documento_limpo = re.compile("<.*?>")
    return re.sub(documento_limpo, "", documento)

Funções auxiliares de arquivos

In [ ]:
def carregar(nome_arquivo, encoding="Windows-1252"):
    """
      Carrega um arquivo texto e retorna as linhas como um único parágrafo(texto).

      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser carregado.
    """

    # Abre o arquivo
    arquivo = open(nome_arquivo, "r", encoding= encoding)

    paragrafo = ""
    for linha in arquivo:
        linha = linha.splitlines()
        linha = " ".join(linha)
        # Remove as tags existentes no final das linhas
        linha = remove_tags(linha)
        if linha != "":
          paragrafo = paragrafo + linha.strip() + " "

    # Fecha o arquivo
    arquivo.close()

    # Remove os espaços em branco antes e depois do parágrafo
    return paragrafo.strip()

In [ ]:
def carregarLista(nome_arquivo, encoding="Windows-1252"):
    """
      Carrega um arquivo texto e retorna as linhas como uma lista de sentenças(texto).

      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser carregado.
        `encoding` - Codificação dos caracteres do arquivo.
    """

    # Abre o arquivo
    arquivo = open(nome_arquivo, "r", encoding= encoding)

    sentencas = []
    for linha in arquivo:
        linha = linha.splitlines()
        linha = " ".join(linha)
        linha = remove_tags(linha)
        if linha != "":
          sentencas.append(linha.strip())

    # Fecha o arquivo
    arquivo.close()

    return sentencas

In [ ]:
def salvar(nome_arquivo,texto):
    """
      Salva um texto em arquivo.

      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser salvo.
        `texto` - Texto a ser salvo.
    """

    arquivo = open(nome_arquivo, "w")
    arquivo.write(str(texto))
    arquivo.close()

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [ ]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))

    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))

Classe(ModeloArgumentosMedida) de definição dos parâmetros do modelo para medida

In [ ]:
# Import das bibliotecas.
from dataclasses import dataclass, field
from typing import Dict, Optional
from typing import List

@dataclass
class ModeloArgumentosMedida:
    max_seq_len: Optional[int] = field(
        default=None,
        metadata={'help': 'max seq len'},
    )
    pretrained_model_name_or_path: str = field(
        default='neuralmind/bert-base-portuguese-cased',
        metadata={'help': 'nome do modelo pré-treinado do BERT.'},
    )
    modelo_spacy: str = field(
        default="pt_core_news_lg",
        metadata={"help": "nome do modelo do spaCy."},
    )
    versao_modelo_spacy: str = field(
        default="-3.2.0",
        metadata={"help": "versão do nome do modelo no spaCy."},
    )
    do_lower_case: bool = field(
        default=False,
        metadata={'help': 'define se o texto do modelo deve ser todo em minúsculo.'},
    )
    output_attentions: bool = field(
        default=False,
        metadata={'help': 'habilita se o modelo retorna os pesos de atenção.'},
    )
    output_hidden_states: bool = field(
        default=False,
        metadata={'help': 'habilita gerar as camadas ocultas do modelo.'},
    )
    use_wandb : bool = field(
        default=True,
        metadata={'help': 'habilita o uso do wandb.'},
    )
    salvar_avaliacao : bool = field(
        default=True,
        metadata={'help': 'habilita o salvamento do resultado da avaliação.'},
    )
    salvar_medicao : bool = field(
        default=False,
        metadata={'help': 'habilita o salvamento da medicao.'},
    )
    usar_mcl_ajustado : bool = field(
        default=False,
        metadata={'help': 'habilita o carragamento de mcl ajustado.'},
    )
    documentos_perturbados: int = field(
        default="1",
        metadata={"help": "Quantidade de documentos a serem perturbados a partir do original."},
    )
    top_k_predicao: int = field(
        default="100",
        metadata={"help": "Quantidade de palavras a serem recuperadas mais próximas da máscara."},
    )
    estrategia_medida: int = field(
        default=0, # 0 - MEAN estratégia média / 1 - MAX  estratégia maior
        metadata={'help': 'Estratégia de cálculo da médida dos embeddings.'},
    )
    equacao_medida: int = field(
        default=0, # 0 - ADJACENTE / 1 - COMBINAÇÃO TODAS / 2 - CONTEXTO
        metadata={'help': 'Equação de cálculo da coerência.'},
    )
    filtro_palavra: int = field(
        default=0, # 0 - Considera todas as palavras das sentenças / 1 - Desconsidera as stopwords / 2 - Considera somente as palavras substantivas
        metadata={'help': 'Define o filtro de palavras das sentenças para gerar os embeddings.'},
    )
    tamanho_janela: int = field(
        default=0,
        metadata={'help': 'Define o tamanho da janela.'},
    )

Biblioteca de limpeza de tela


In [ ]:
# Import das bibliotecas.
from IPython.display import clear_output

## 1.3 Tratamento de logs

In [ ]:
# Import das bibliotecas.
import logging # Biblioteca de logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s")

logger = logging.getLogger()
logger.setLevel(logging.INFO)

## 1.4  Identificando o ambiente Colab

In [ ]:
# Import das bibliotecas.
import sys # Biblioteca para acessar módulos do sistema

# Se estiver executando no Google Colaboratory
# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

## 1.5 Colaboratory

Usando Colab GPU para Treinamento


Uma GPU pode ser adicionada acessando o menu e selecionando:

`Edit -> Notebook Settings -> Hardware accelerator -> (GPU)`

Em seguida, execute a célula a seguir para confirmar que a GPU foi detectada.

In [ ]:
# Import das bibliotecas.
import tensorflow as tf

# Recupera o nome do dispositido da GPU.
device_name = tf.test.gpu_device_name()

# O nome do dispositivo deve ser parecido com o seguinte:
if device_name == "/device:GPU:0":
    logging.info("Encontrei GPU em: {}".format(device_name))
else:
    logging.info("Dispositivo GPU não encontrado")
    #raise SystemError("Dispositivo GPU não encontrado")

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:root:Dispositivo GPU não encontrado


Nome da GPU

Para que a torch use a GPU, precisamos identificar e especificar a GPU como o dispositivo. Posteriormente, em nosso ciclo de treinamento, carregaremos dados no dispositivo.

Vale a pena observar qual GPU você recebeu. A GPU Tesla P100 é muito mais rápido que as outras GPUs, abaixo uma lista ordenada:
- 1o Tesla P100
- 2o Tesla T4
- 3o Tesla P4 (Não tem memória para execução 4 x 8, somente 2 x 4)
- 4o Tesla K80 (Não tem memória para execução 4 x 8, somente 2 x 4)

In [ ]:
# Import das bibliotecas.
import torch

def getDeviceGPU():
    """
      Retorna um dispositivo de GPU se disponível ou CPU.

      Retorno:
        `device` - Um device de GPU ou CPU.
    """

    # Se existe GPU disponível.
    if torch.cuda.is_available():

        # Diz ao PyTorch para usar GPU.
        device = torch.device("cuda")

        logging.info("Existem {} GPU(s) disponíveis.".format(torch.cuda.device_count()))
        logging.info("Iremos usar a GPU: {}.".format(torch.cuda.get_device_name(0)))

    # Se não.
    else:
        logging.info("Sem GPU disponível, usando CPU.")
        device = torch.device("cpu")

    return device

In [ ]:
# Recupera o device com GPU ou CPU
device = getDeviceGPU()

INFO:root:Sem GPU disponível, usando CPU.


Memória

Memória disponível no ambiente

In [ ]:
# Importando as bibliotecas.
from psutil import virtual_memory

ram_gb = virtual_memory().total / 1e9
logging.info("Seu ambiente de execução tem {: .1f} gigabytes de RAM disponível\n".format(ram_gb))

if ram_gb < 20:
  logging.info("Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> \"Alterar tipo de tempo de execução\"")
  logging.info("e selecione High-RAM. Então, execute novamente está célula")
else:
  logging.info("Você está usando um ambiente de execução de memória RAM alta!")

INFO:root:Seu ambiente de execução tem  13.6 gigabytes de RAM disponível

INFO:root:Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> "Alterar tipo de tempo de execução"
INFO:root:e selecione High-RAM. Então, execute novamente está célula


## 1.6 Monta uma pasta no google drive para carregar os arquivos de dados.

In [ ]:
# import necessário
from google.colab import drive

# Monta o drive na pasta especificada
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1.7 Instalação do wandb

Instalação

In [ ]:
!pip install --upgrade wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 1.8 Instalação do spaCy

https://spacy.io/

Modelos do spaCy para português:
https://spacy.io/models/pt

In [ ]:
# Instala o spacy
!pip install -U pip setuptools wheel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Instala uma versão específica
!pip install -U spacy==3.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 1.9 Instalação do Gensim

Instalando o gensim no Google Colaboratory.

No Jupiter Notebook executar através "Anaconda Prompt".


In [ ]:
#!pip install -U gensim
!pip install -U gensim==4.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 2 Parametrização

## Gerais

In [ ]:
# Nome base das saidas do projeto
NOME_BASE_SAIDA = "MedidaTopicoCohQuADInen_v1"

# Definição dos parâmetros a serem avaliados
#Quantidade de documentos a serem perturbados a partir do original.
DOCUMENTOS_PERTURBADOS = 1

#Quantidade de palavras a serem recuperadas mais próximas da máscara.
TOP_K_PREDICAO = 1

# Filtro de palavras das sentenças[0,1,2,3,4,5] 'TAP,SSW,SVS,SP_TAP,SP_SSW,SP_SVS'
FILTRO_PALAVRAS_STR = ["TODAS_AS_PALAVRAS",
                       "SEM_STOPWORDS",
                       "SOMENTE_VERBOS_SUBSTANTIVOS",
                       "SEM_PONTUACAO_TODAS_AS_PALAVRAS",
                       "SEM_PONTUACAO_SEM_STOPWORDS",
                       "SEM_PONTUACAO_SOMENTE_VERBOS_SUBSTANTIVOS",
                       ]

# FILTRO_PALAVRAS_STR_ABREV = ["TAP","SSW","SVS","SP_TAP","SP_SSW","SP_SVS"]
# FILTRO_PALAVRAS = [0, 1, 2, 3, 4, 5]

FILTRO_PALAVRAS_STR_ABREV = ["SP_SSW","SP_SVS"]
FILTRO_PALAVRAS = [3, 4, 5]

# Tamanho de janelas a serem avaliados
TAMANHO_JANELA_INICIO = 1
TAMANHO_JANELA_FIM = 150

## Específicos

Parâmetros do modelo

In [ ]:
# Definição dos parâmetros do Modelo
model_args = ModeloArgumentosMedida(
    modelo_spacy = "en_core_web_lg",
    #modelo_spacy = "en_core_web_md",
    #modelo_spacy = "en_core_web_sm",
    versao_modelo_spacy = "3.2.0",
    do_lower_case = True,  # default True
    use_wandb = True,
    salvar_medicao = True, #Salva o resultado da medição
    salvar_avaliacao = True, # Salva o resultado da avaliação das medições
    documentos_perturbados = DOCUMENTOS_PERTURBADOS, # Quantidade de documentos a serem perturbados a partir do original.
    top_k_predicao = TOP_K_PREDICAO, # Conjunto de valores: 1, 10, 100, 500 e 1000. Quantidade de palavras a serem recuperadas mais próximas da máscara.
    equacao_medida = 0, # Atributo usado para os logs do wandb. 0 - Palavras adjacentes / 1 - Todas as palavras / 2 - Palavra e contexto
    filtro_palavra = 0, # # Atributo usado para os logs do wandb. 0 - Considera todas as palavras das sentenças / 1 - Desconsidera as stopwords / 2 - Considera somente as palavras substantivas
    tamanho_janela = 0 # Atributo usado para os logs do wandb.
)

## Nome do diretório dos arquivos de dados

In [ ]:
# Diretório do cohebert
DIRETORIO_COHEBERT = "COHQUAD_IN_EN"

## Define o caminho para os arquivos de dados

In [ ]:
# Diretório local para os arquivos pré-processados
DIRETORIO_LOCAL = "/content/" + DIRETORIO_COHEBERT + "/"

# Diretório no google drive com os arquivos pré-processados
DIRETORIO_DRIVE = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_COHEBERT + "/"

## Inicialização diretórios

Diretório base local

In [ ]:
# Importando as bibliotecas.
import os

def criaDiretorioLocal():

  # Cria o diretório para receber os arquivos Originais e Permutados
  # Diretório a ser criado
  dirbase = DIRETORIO_LOCAL[:-1]

  if not os.path.exists(dirbase):
      # Cria o diretório
      os.makedirs(dirbase)
      logging.info("Diretório criado: {}.".format(dirbase))
  else:
      logging.info("Diretório já existe: {}.".format(dirbase))

In [ ]:
criaDiretorioLocal()

INFO:root:Diretório já existe: /content/COHQUAD_IN_EN.


Diretório para conter as os resultados das medidas

In [ ]:
# Import de bibliotecas.
import os

def criaDiretorioMedidacao():
  DIRETORIO_BASE = DIRETORIO_DRIVE + "validacao_medicao_topico_palavra"

  # Verifica se o diretório existe
  if not os.path.exists(DIRETORIO_BASE):
    # Cria o diretório
    os.makedirs(DIRETORIO_BASE)
    logging.info("Diretório criado: {}.".format(DIRETORIO_BASE))
  else:
    logging.info("Diretório já existe: {}.".format(DIRETORIO_BASE))

In [ ]:
criaDiretorioMedidacao()

INFO:root:Diretório já existe: /content/drive/MyDrive/Colab Notebooks/Data/COHQUAD_IN_EN/validacao_medicao_topico_palavra.


Diretório para conter os arquivos da avaliação

In [ ]:
# Import de bibliotecas.
import os

def criaDiretorioAvaliacao():
  DIRETORIO_BASE = DIRETORIO_DRIVE + "validacao_medicao_topico_palavra/Avaliacao"

  # Verifica se o diretório existe
  if not os.path.exists(DIRETORIO_BASE):
    # Cria o diretório
    os.makedirs(DIRETORIO_BASE)
    logging.info("Diretório criado: {}.".format(DIRETORIO_BASE))
  else:
    logging.info("Diretório já existe: {}.".format(DIRETORIO_BASE))

In [ ]:
criaDiretorioAvaliacao()

INFO:root:Diretório já existe: /content/drive/MyDrive/Colab Notebooks/Data/COHQUAD_IN_EN/validacao_medicao_topico_palavra/Avaliacao.


Diretório para conter os arquivos das medidas

In [ ]:
# Import de bibliotecas.
import os

def criaDiretorioMedicao():

  DIRETORIO_BASE = DIRETORIO_DRIVE + "validacao_medicao_topico_palavra/Medicao"

  # Verifica se o diretório existe
  if not os.path.exists(DIRETORIO_BASE):
    # Cria o diretório
    os.makedirs(DIRETORIO_BASE)
    logging.info("Diretório criado: {}.".format(DIRETORIO_BASE))
  else:
    logging.info("Diretório já existe: {}.".format(DIRETORIO_BASE))

In [ ]:
criaDiretorioMedicao()

INFO:root:Diretório já existe: /content/drive/MyDrive/Colab Notebooks/Data/COHQUAD_IN_EN/validacao_medicao_topico_palavra/Medicao.


# 3 spaCy

## 3.1 Download arquivo modelo

https://spacy.io/models/pt

### Função download modelo spaCy

In [ ]:
def downloadSpacy(model_args):
    """
      Realiza o download do arquivo do modelo para o diretório corrente.

      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.
    """
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()

    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy
    # Nome arquivo compactado
    NOME_ARQUIVO_MODELO_COMPACTADO = ARQUIVO_MODELO_SPACY + VERSAO_SPACY + ".tar.gz"

    # Url do arquivo
    URL_ARQUIVO_MODELO_COMPACTADO = "https://github.com/explosion/spacy-models/releases/download/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/" + NOME_ARQUIVO_MODELO_COMPACTADO

    # Realiza o download do arquivo do modelo
    logging.info("Download do arquivo do modelo do spaCy.")
    downloadArquivo(URL_ARQUIVO_MODELO_COMPACTADO, DIRETORIO_COHEBERT + "/" + NOME_ARQUIVO_MODELO_COMPACTADO)

## 3.2 Descompacta o arquivo do modelo

### Função descompacta modelo spaCy

In [ ]:
# Import das bibliotecas.
import tarfile # Biblioteca de descompactação

def descompactaSpacy(model_args):
    """
      Descompacta o arquivo do modelo.

      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.
    """

    # Verifica se existe o diretório base do cohebert e retorna o nome do diretório
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()

    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy

    # Nome do arquivo a ser descompactado
    NOME_ARQUIVO_MODELO_COMPACTADO = DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + ".tar.gz"

    logging.info("Descompactando o arquivo do modelo do spaCy.")
    arquivoTar = tarfile.open(NOME_ARQUIVO_MODELO_COMPACTADO, "r:gz")
    arquivoTar.extractall(DIRETORIO_COHEBERT)
    arquivoTar.close()

    # Apaga o arquivo compactado
    if os.path.isfile(NOME_ARQUIVO_MODELO_COMPACTADO):
        os.remove(NOME_ARQUIVO_MODELO_COMPACTADO)

## 3.3 Carrega o modelo

### Função carrega modelo spaCy

In [ ]:
# Import das bibliotecas.
import spacy # Biblioteca do spaCy

def carregaSpacy(model_args):
    """
    Realiza o carregamento do Spacy.

    Parâmetros:
      `model_args` - Objeto com os argumentos do modelo.
    """

    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()

    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy
    # Caminho raoz do modelo do spaCy
    DIRETORIO_MODELO_SPACY =  DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY

    # Verifica se o diretório existe
    if os.path.exists(DIRETORIO_MODELO_SPACY) == False:
        # Realiza o download do arquivo modelo do spaCy
        downloadSpacy(model_args)
        # Descompacta o spaCy
        descompactaSpacy(model_args)

    # Diretório completo do spaCy
    DIRETORIO_MODELO_SPACY = DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/" + ARQUIVO_MODELO_SPACY + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/"

    # Carrega o spaCy. Necessário somente "tagger" para encontrar os substantivos
    nlp = spacy.load(DIRETORIO_MODELO_SPACY)
    logging.info("spaCy carregado.")

    # Retorna o spacy carregado
    return nlp

### Carrega o modelo spaCy


In [ ]:
# Carrega o modelo spaCy
nlp = carregaSpacy(model_args)

INFO:root:spaCy carregado.


## 3.4 Funções auxiliares spaCy

### getStopwords

Recupera as stopwords do spaCy

In [ ]:
def getStopwords(nlp):
    """
      Recupera as stop words do nlp(Spacy).

      Parâmetros:
        `nlp` - Um modelo spaCy carregado.
    """

    spacy_stopwords = nlp.Defaults.stop_words

    return spacy_stopwords

Lista dos stopwords

In [ ]:
logging.info("Quantidade de stopwords: {}.".format(len(getStopwords(nlp))))

print(getStopwords(nlp))

INFO:root:Quantidade de stopwords: 326.


{'most', 'while', 'otherwise', 'whom', 'bottom', 'empty', 'been', 'the', 'everyone', 'than', 'very', 'others', 'elsewhere', 'fifty', 'three', 'please', 'though', 'few', 'anything', 'say', 'various', 'their', 'more', '’s', 'then', 'always', 'off', 'really', 'before', 'become', 'myself', 'ever', 'they', '‘ve', 'sometime', '‘d', 'during', 'further', 'give', 'nevertheless', 'some', 'whole', 'it', 'whether', 'its', 'what', 'you', 'of', 'there', 'when', 'without', 'already', '’d', 'latter', 'per', 'often', 'by', 'anywhere', 'yourself', 'whose', 'up', 'can', 'doing', "n't", 'with', 'even', '’ll', 'nine', 'never', 'using', 'show', 'hundred', 'but', 'ourselves', 'call', 'this', 'top', 'six', 'my', 'except', 'first', 'namely', 'am', 'n‘t', 'each', 'serious', 'ten', 'themselves', 'since', 'afterwards', 'how', 'whither', 'beyond', 'we', 'something', 'has', 'twenty', 'toward', 'quite', 'yourselves', 'mostly', 'around', 'put', 'sixty', 'did', 'someone', 'beside', 'once', 'own', 'just', "'s", 'those'

### getVerbos
Localiza os verbos da sentença

In [ ]:
# Import das bibliotecas.
import spacy
from spacy.util import filter_spans
from spacy.matcher import Matcher

# (verbo normal como auxilar ou auxilar) + vários verbos auxiliares +verbo principal ou verbo auxiliar
gramaticav1 =  [
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["aux","aux:pass"]}},  #verbo auxiliar
                {"POS": "VERB", "OP": "?", "DEP": {"IN": ["ROOT","aux","xcomp","aux:pass"]}},  #verbo normal como auxiliar
                {"POS": "AUX", "OP": "*", "DEP": {"IN": ["aux","xcomp","aux:pass"]}},  #verbo auxiliar
                {"POS": "VERB", "OP": "+"}, #verbo principal
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["cop","aux","xcomp","aux:pass"]}},  #verbo auxiliar
               ]

# verbo auxiliar + verbo normal como auxiliar + conjunção com preposição + verbo
gramaticav2 =  [
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["aux","aux:pass"]}},  #verbo auxiliar
                {"POS": "VERB", "OP": "+", "DEP": {"IN": ["ROOT"]}},  #verbo principal
                {"POS": "SCONJ", "OP": "+", "DEP": {"IN": ["mark"]}}, #conjunção com preposição
                {"POS": "VERB", "OP": "+", "DEP": {"IN": ["xcomp"]}}, #verbo normal como complementar
               ]

#Somente verbos auxiliares
gramaticav3 =  [
                {"POS": "AUX", "OP": "?"},  #Verbos auxiliar
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["cop"]}},  #Verbos auxiliar de ligação (AUX+(cop))
                {"POS": "ADJ", "OP": "+", "DEP": {"IN": ["ROOT"]}},
                {"POS": "AUX", "OP": "?"}  #Verbos auxiliar
               ]

matcherv = Matcher(nlp.vocab)

matcherv.add("frase verbal", [gramaticav1])
matcherv.add("frase verbal", [gramaticav2])
matcherv.add("frase verbal", [gramaticav3])

#Retorna a Frase Verbal
def getVerbos(periodo):
  #Processa o período
  doc1 = nlp(periodo.text)

  # Chama o mather para encontrar o padrão
  matches = matcherv(doc1)

  padrao = [doc1[start:end] for _, start, end in matches]

  #elimina as repetições e sobreposições
  #return filter_spans(padrao)
  lista1 = filter_spans(padrao)

  # Converte os itens em string
  lista2 = []
  for x in lista1:
      lista2.append(str(x))

  return lista2

### getDicPOSQtde

Conta as POS Tagging de uma sentença

In [ ]:
def getDicPOSQtde(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Retorna inteiros que mapeiam para classes gramaticais
  conta_dicionarios = doc.count_by(spacy.attrs.IDS["POS"])

  # Dicionário com as tags e quantidades
  novodic = dict()

  for pos, qtde in conta_dicionarios.items():
    classe_gramatical = doc.vocab[pos].text
    novodic[classe_gramatical] = qtde

  return novodic

In [ ]:
def getDicTodasPOSQtde(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Retorna inteiros que mapeiam para classes gramaticais
  conta_dicionarios = doc.count_by(spacy.attrs.IDS["POS"])

  # Dicionário com as tags e quantidades
  novodic = {"PRON":0, "VERB":0, "PUNCT":0, "DET":0, "NOUN":0, "AUX":0, "CCONJ":0, "ADP":0, "PROPN":0, "ADJ":0, "ADV":0, "NUM":0, "SCONJ":0, "SYM":0, "SPACE":0, "INTJ":0, "X": 0}

  for pos, qtde in conta_dicionarios.items():
    classe_gramatical = doc.vocab[pos].text
    novodic[classe_gramatical] = qtde

  return novodic

### getDicTodasPOSQtde

Conta as POS Tagging de uma sentença

In [ ]:
def getDicTodasPOSQtde(lista):

  # Dicionário com as tags e quantidades
  conjunto = {"PRON":0, "VERB":0, "PUNCT":0, "DET":0, "NOUN":0, "AUX":0, "CCONJ":0, "ADP":0, "PROPN":0, "ADJ":0, "ADV":0, "NUM":0, "SCONJ":0, "SYM":0, "SPACE":0, "INTJ": 0}

  for x in lista:
    valor = conjunto.get(x)
    if valor != None:
      conjunto[x] = valor + 1
    else:
      conjunto[x] = 1

  return conjunto

### getSomaDic

Soma os valores de dicionários com as mesmas chaves.

In [ ]:
from collections import Counter
from functools import reduce

def atualizaValor(a,b):
    a.update(b)
    return a

def getSomaDic(lista):

  # Soma os dicionários da lista
  novodic = reduce(atualizaValor, (Counter(dict(x)) for x in lista))

  return novodic

### getTokensSentenca

Retorna a lista de tokens da sentenca.

In [ ]:
def getTokensSentenca(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  lista = []

  # Percorre a sentença adicionando os tokens
  for token in doc:
    lista.append(token.text)

  return lista

### getPOSTokensSentenca

Retorna a lista das POS-Tagging dos tokens da sentenca.

In [ ]:
def getPOSTokensSentenca(sentenca):

  # Verifica se o sentenca não foi processado pelo spaCy
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  lista = []

  # Percorre a sentença adicionando os tokens
  for token in doc:
    lista.append(token.pos_)

  return lista

### getListaTokensPOSSentenca

Retorna duas listas uma com os tokens e a outra com a POS-Tagging dos tokens da sentenca.

In [ ]:
def getListaTokensPOSSentenca(sentenca):
  # Verifica se o sentenca não foi processado pelo spaCy
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  listatokens = []
  listapos = []

  # Percorre a sentença adicionando os tokens e as POS
  for token in doc:
    listatokens.append(token.text)
    listapos.append(token.pos_)

  return listatokens, listapos

### Tradução das tags

Tags de palavras universal

https://universaldependencies.org/u/pos/

Detalhes das tags em português:
http://www.dbd.puc-rio.br/pergamum/tesesabertas/1412298_2016_completo.pdf

In [ ]:
#dicionário que contêm pos tag universal e suas explicações
palavra_universal_dict = {
  "X"    : "Outro",
  "VERB" : "Verbo ",
  "SYM"  : "Símbolo",
  "CONJ" : "Conjunção",
  "SCONJ": "Conjunção subordinativa",
  "PUNCT": "Pontuação",
  "PROPN": "Nome próprio",
  "PRON" : "Pronome substativo",
  "PART" : "Partícula, morfemas livres",
  "NUM"  : "Numeral",
  "NOUN" : "Substantivo",
  "INTJ" : "Interjeição",
  "DET"  : "Determinante, Artigo e pronomes adjetivos",
  "CCONJ": "Conjunção coordenativa",
  "AUX"  : "Verbo auxiliar",
  "ADV"  : "Advérbio",
  "ADP"  : "Preposição",
  "ADJ"  : "Adjetivo"
}

#Explica a POS
def getPOSPalavraUniversalTraduzido(palavra):
  if palavra in palavra_universal_dict.keys():
      traduzido = palavra_universal_dict[palavra]
  else:
      traduzido = "NA"
  return traduzido

### getSentencaSemStopWord

Retorna uma lista dos tokens sem as stopwords.

In [ ]:
def getSentencaSemStopWord(sentenca, stopwords):

  # Lista dos tokens
  lista = []

  # Percorre os tokens da sentença
  for i, token in enumerate(sentenca):

    # Verifica se o token é uma stopword
    if token.lower() not in stopwords:
      lista.append(token)

  # Retorna o documento
  return lista

### getSentencaSalientePOS

Retorna uma lista das palavras do tipo especificado.

In [ ]:
def getSentencaSalientePOS(sentenca, pos, classe_saliente=["NOUN"]):

  # Lista dos tokens
  lista = []

  # Percorre a sentença
  for i, token in enumerate(sentenca):

    # Verifica se o token é do tipo especificado
    if pos[i] in classe_saliente:
      lista.append(token)

  # Retorna o documento
  return lista

###removeStopWords

Remove as stopwords de um documento ou senteça.

In [ ]:
def removeStopWord(documento, stopwords):

  # Remoção das stopwords do documento
  documentoSemStopwords = [palavra for palavra in documento.split() if palavra.lower() not in stopwords]

  # Concatena o documento sem os stopwords
  documento_limpo = " ".join(documentoSemStopwords)

  # Retorna o documento
  return documento_limpo

### getTokensSemStopword

Retira as stopswords de lista de tokens

In [ ]:
def getTokensSemStopword(tokens, spacy_stopwords=getStopwords(nlp)):
    """
      Retira os tokens da lista de tokens tokens que estão na lista de stopword.
      A lista de tokens pode ou não estar dentro de uma outra lista.

      Parâmetros:
        `tokens` - Uma lista com os tokens ou uma lista de lista de tokens.
        `spacy_stopwords` - Uma lista com as stopword.
    """

    # Verifica se é uma lista de palavras(str) ou ou uma lista de lista
    if type(tokens[0]) is str:
      lista_tokens = [tokens]
    else:
      lista_tokens = tokens

    # Lista de retorno
    lista_tokens_sem_stopwords = []

    # Percorre a lista de tokens
    for texto in lista_tokens:

      # Lista dos tokens sem as stopwords
      tokens_sem_stopwords = []

      # Percorre os tokens
      for token in texto:
        # Verifica se o toke não está na lista de stopwords para adicionar a nova lista
        if token not in spacy_stopwords:
          tokens_sem_stopwords.append(token)

      # Adiciona a lista de tokens sem stopwords na lista de retorno se tiver uma palavra
      if len(tokens_sem_stopwords) != 0:
        lista_tokens_sem_stopwords.append(tokens_sem_stopwords)

    if type(tokens[0]) is str:
      return lista_tokens_sem_stopwords[0]
    else:
      return lista_tokens_sem_stopwords

### getSentencasTexto

Retorna a lista de tokens de uma lista de textos.

In [ ]:
def getSentencasTexto(textos, nlp = nlp):

  """
     Sentencia um texto ou uma lista de textos.

     Parâmetros:
      `textos` - Um texto(str) ou uma lista de textos.
      `nlp` - Modelo spacy carregado.

  """

  # Verifica se é um texto é str ou uma lista de texto
  if type(textos) is str:
    lista_texto = [textos]
  else:
    lista_texto = textos

  # Lista dos tokens
  lista_sentencas = []

  for texto in lista_texto:

    # Sentencia o documento
    doc = nlp(texto)

    # Percorre as sentenças do documento
    for sentenca in doc.sents:

        lista_sentencas.append(str(sentenca))

  # Verifica o tipo documento para o tipo de retorno
  if type(textos) is str:
    return lista_sentencas[0]
  else:
    return lista_sentencas

### getSentencasMinusculo

Retorna a lista das sentencas do texto em minúsculo.

In [ ]:
def getSentencasMinusculo(textos):

  """
     Sentencia um texto ou uma lista de textos em minusculo.

     Parâmetros:
      `textos` - Um texto(str) ou uma lista de textos.

  """

  # Verifica se é um texto é str ou uma lista de texto
  if type(textos) is str:
    lista_texto = [textos]
  else:
    lista_texto = textos

  # Lista dos tokens
  lista_sentencas = []

  for texto in lista_texto:

    lista_sentencas.append(str(texto).lower())

  # Verifica o tipo documento para o tipo de retorno
  if type(textos) is str:
    return lista_sentencas[0]
  else:
    return lista_sentencas

### getTokensTexto

Retorna a lista de tokens do texto.

In [ ]:
def getTokensTexto(textos, nlp = nlp):

  """
     Tokeniza um texto ou uma lista de textos.

     Parâmetros:
      `textos` - Um texto(str) ou uma lista de textos.
  """

  # Verifica se é um texto é str ou uma lista de texto
  if type(textos) is str:
    lista_texto = [textos]
  else:
    lista_texto = textos

  # Lista de retorno
  lista_tokens_texto = []

  # Percorre a lista de texto
  for texto in lista_texto:

    # Verifica se o sentenca não foi processado pelo spaCy
    if type(texto) is not spacy.tokens.doc.Doc:
        # Realiza o parsing no spacy
        doc = nlp(texto)
    else:
        doc = texto

    # Lista dos tokens
    lista_tokens = []

    # Percorre a sentença adicionando os tokens
    for token in doc:
      lista_tokens.append(token.text)

    # Adiciona a lista de tokens na lista de sentenças
    lista_tokens_texto.append(lista_tokens)

  # Verifica o tipo documento para o tipo de retorno
  if type(textos) is str:
    return lista_tokens_texto[0]
  else:
    return lista_tokens_texto

### removerPontuacao

Remove pontuação

In [ ]:
def removerPontuacao(textos):

    """https://spacy.io/api/annotation"""

    textos_saida = []

    for texto in textos:

        doc = nlp(" ".join(texto))

        sentenca = []
        for token in doc:
          if token.pos_ not in ['PUNCT']:
              sentenca.append(token.text)

        if len(sentenca) != 0:
          textos_saida.append(sentenca)

    return textos_saida

### relevantes

Palavras relevantes

In [ ]:
def relevantes(textos, postags_permitidas=['VER', 'AUX', 'NOUN']):

    """https://spacy.io/api/annotation"""

    textos_saida = []

    for texto in textos:

        doc = nlp(" ".join(texto))

        sentenca = []
        for token in doc:
          if token.pos_ in postags_permitidas:
              sentenca.append(token.text)

        if len(sentenca) != 0:
          textos_saida.append(sentenca)

    return textos_saida

### lematizacao

Lematização do texto

In [ ]:
def lematizacao(textos, postags_permitidas=['NOUN', 'ADJ', 'VERB', 'ADV']):

    """https://spacy.io/api/annotation"""

    textos_saida = []

    for texto in textos:
        doc = nlp(" ".join(texto))

        sentenca = []
        for token in doc:
          if token.pos_ in postags_permitidas:
              sentenca.append(token.lemma_)

        if len(sentenca) != 0:
          textos_saida.append(sentenca)

    return textos_saida

### preparaCorpus

In [ ]:
# Import das biblitecas
import pandas as pd
import re
import gensim

def preparaCorpus(textos,
                  sentenciaTexto=False,
                  tornaMinusculo=False,
                  removePontuacao=False,
                  removeStopwords=False,
                  bigramas=False,
                  trigramas=False,
                  somenteRelevante=False,
                  postag_relevante=['VERB', 'AUX', 'NOUN'],
                  lematizar=False,
                  postag_lema=['NOUN', 'ADJ', 'VERB', 'ADV']):

    # Verifica se é um textos é str ou uma lista de texto
    if type(textos) is str:
      # Sentencia o texto
      lista_sentencas = [textos]
    else:
      lista_sentencas = textos

    # Converte o texto em uma lista de sentencas
    if sentenciaTexto==True:
      lista_sentencas = getSentencasTexto(lista_sentencas)

    # Converte o texto em minúsuclo
    if tornaMinusculo==True:
      lista_sentencas = getSentencasMinusculo(lista_sentencas)

    # tokeniza o texto
    lista_sentencas_palavras = getTokensTexto(lista_sentencas)

    # Remove a pontuação
    if removePontuacao==True:
        lista_sentencas_palavras = removerPontuacao(lista_sentencas_palavras)

    # Remove as stop words
    if removeStopwords==True:
      lista_sentencas_palavras = getTokensSemStopword(lista_sentencas_palavras)

    # Criar bigramas ou trigramas
    if bigramas==True:
      # Construa os modelos de bigramas
      bigram = gensim.models.Phrases(lista_sentencas_palavras, min_count=5, threshold=100) # max_topicse mais alto menos frases.
      # Maneira mais rápida de obter uma frase batida como um trigrama/bigrama
      bigram_mod = gensim.models.phrases.Phraser(bigram)
      lista_sentencas_palavras = [bigram_mod[doc] for doc in lista_sentencas_palavras]

    if trigramas==True:
      # Construa os modelos de bigramas
      bigram = gensim.models.Phrases(lista_sentencas_palavras, min_count=5, threshold=100) # max_topicse mais alto menos frases.
      # Maneira mais rápida de obter uma frase batida como um trigrama/bigrama
      bigram_mod = gensim.models.phrases.Phraser(bigram)
      # Construa os modelos de trigramas
      trigram = gensim.models.Phrases(bigram[lista_sentencas_palavras], threshold=100)
      # Maneira mais rápida de obter uma frase batida como um trigrama/bigrama
      trigram_mod = gensim.models.phrases.Phraser(trigram)
      lista_sentencas_palavras = [trigram_mod[bigram_mod[doc]] for doc in lista_sentencas_palavras]

    # Somente palavras relevantes
    if somenteRelevante==True:
      lista_sentencas_palavras = relevantes(lista_sentencas_palavras, postags_permitidas=postag_relevante)

    # Faça a lematização mantendo apenas para noun, adj, vb, adv
    if lematizar==True:
      lista_sentencas_palavras = lematizacao(lista_sentencas_palavras, postags_permitidas=postag_lema)

    return lista_sentencas_palavras

# 4 Funções auxiliares

## concatenaListas

In [ ]:
def concatenaListas(lista, pos=1):
  lista_concat = []

  for x in lista:
      lista_concat = lista_concat + x[pos]

  return lista_concat

## encontrarIndiceSubLista

Retorna os índices de início e fim da sublista na lista

In [ ]:
# Localiza os índices de início e fim de uma sublista em uma lista
def encontrarIndiceSubLista(lista, sublista):

    """
      Localiza os índices de início e fim de uma sublista em uma lista.

      Parâmetros:
      `lista` - Uma lista.
      `sublista` - Uma sublista a ser localizada na lista.
    """
    # https://en.wikipedia.org/wiki/Boyer%E2%80%93Moore%E2%80%93Horspool_algorithm

    # Recupera o tamanho da lista
    h = len(lista)
    # Recupera o tamanho da sublista
    n = len(sublista)
    skip = {sublista[i]: n - i - 1 for i in range(n - 1)}
    i = n - 1
    while i < h:
      for j in range(n):
        if lista[i - j] != sublista[-j - 1]:
            i += skip.get(lista[i], n)
            break
        else:
            indice_inicio = i - n + 1
            indice_fim = indice_inicio + len(sublista)-1

            return indice_inicio, indice_fim

    # Não encontrou a sublista na lista
    return -1, -1

# 5 Comparar documentos





## 5.1 Carregamento dos arquivos de dados originais e perturbados

#### 5.1.1 Especifica os nomes dos arquivos de dados



In [ ]:
# Nome do arquivo
NOME_ARQUIVO_ORIGINAL = "original.csv"
NOME_ARQUIVO_ORIGINAL_COMPACTADO = "original.zip"
NOME_ARQUIVO_ORIGINAL_POS = "originalpos.csv"
NOME_ARQUIVO_ORIGINAL_POS_COMPACTADO = "originalpos.zip"

NOME_ARQUIVO_PERTURBADO = "perturbado_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".csv"
NOME_ARQUIVO_PERTURBADO_COMPACTADO = "perturbado_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".zip"
NOME_ARQUIVO_PERTURBADO_POS = "perturbadopos_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".csv"
NOME_ARQUIVO_PERTURBADO_POS_COMPACTADO = "perturbadopos_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".zip"

### 5.1.2 Cria o diretório local para receber os dados

In [ ]:
# Importando as bibliotecas.
import os

# Cria o diretório para receber os arquivos Originais e Permutados
# Diretório a ser criado
dirbase = DIRETORIO_LOCAL[:-1]

if not os.path.exists(dirbase):
    # Cria o diretório
    os.makedirs(dirbase)
    logging.info("Diretório criado: {}.".format(dirbase))
else:
    logging.info("Diretório já existe: {}.".format(dirbase))

INFO:root:Diretório já existe: /content/COHQUAD_IN_EN.


### 5.1.3 Copia os arquivos do Google Drive para o Colaboratory

In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_ORIGINAL_COMPACTADO" "$DIRETORIO_LOCAL"
  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_ORIGINAL_POS_COMPACTADO" "$DIRETORIO_LOCAL"

  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_PERTURBADO_COMPACTADO" "$DIRETORIO_LOCAL"
  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_PERTURBADO_POS_COMPACTADO" "$DIRETORIO_LOCAL"

  logging.info("Terminei a cópia.")

INFO:root:Terminei a cópia.


Descompacta os arquivos

Usa o unzip para descompactar:
*   `-o` sobrescreve o arquivo se existir
*   `-j` Não cria nenhum diretório
*   `-q` Desliga as mensagens
*   `-d` Diretório de destino


In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:
  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_ORIGINAL_COMPACTADO" -d "$DIRETORIO_LOCAL"
  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_ORIGINAL_POS_COMPACTADO" -d "$DIRETORIO_LOCAL"

  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_PERTURBADO_COMPACTADO" -d "$DIRETORIO_LOCAL"
  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_PERTURBADO_POS_COMPACTADO" -d "$DIRETORIO_LOCAL"

  logging.info("Terminei a descompactação.")

INFO:root:Terminei a descompactação.


### 5.1.4 Carregamento das lista com os dados dos arquivos originais e pertubados

#### Carrega o arquivo dos dados originais e POS

In [ ]:
# Import das bibliotecas.
import pandas as pd

# Abre o arquivo e retorna o DataFrame
lista_documentos_originais = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_ORIGINAL, sep=";", encoding="UTF-8")
lista_documentos_originais_pos = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_ORIGINAL_POS, sep=";", encoding="UTF-8")

logging.info("TERMINADO ORIGINAIS: {}.".format(len(lista_documentos_originais)))
logging.info("TERMINADO ORIGINAIS POS: {}.".format(len(lista_documentos_originais_pos)))

INFO:root:TERMINADO ORIGINAIS: 20.
INFO:root:TERMINADO ORIGINAIS POS: 20.


In [ ]:
lista_documentos_originais.sample(5)

,id,sentencas,documento
12,13p0,['What is a stack and how to enqueue an elemen...,What is a stack and how to enqueue an element ...
2,3p0,['How to push elements in a queue?'],How to push elements in a queue?
1,2p0,['How to dequeue elements in a stack?'],How to dequeue elements in a stack?
13,14p0,['What is a queue and how to push an element o...,What is a queue and how to push an element on it?
4,5p0,['How to push elements in a queue data structu...,How to push elements in a queue data structure?


In [ ]:
# Corrige os tipos dos dados da lista agrupada
tipos = {"id": str}

lista_documentos_originais = lista_documentos_originais.astype(tipos)

In [ ]:
lista_documentos_originais_pos.sample(5)

,id,pos_documento
3,4p0,"[[['How', 'to', 'push', 'and', 'pop', 'element..."
1,2p0,"[[['How', 'to', 'dequeue', 'elements', 'in', '..."
13,14p0,"[[['What', 'is', 'a', 'queue', 'and', 'how', '..."
19,20p0,"[[['In', 'a', 'queue', 'does', 'the', 'push', ..."
7,8p0,"[[['How', 'to', 'pop', 'elements', 'in', 'a', ..."


In [ ]:
# Corrige os tipos dos dados da lista agrupada
tipos = {"id": str}

lista_documentos_originais_pos = lista_documentos_originais_pos.astype(tipos)

#### Corrigir os tipos de colunas dos dados originais e POS

Em dados originais:
- coluna 1 - `sentenças` carregadas do arquivo vem como string e não como lista.

Em dados originais pos:
- coluna 1 - `pos_documento` carregadas do arquivo vem como string e não como lista.

In [ ]:
# Import das bibliotecas.
import ast # Biblioteca para conversão de string em lista

# Verifica se o tipo da coluna não é list e converte
lista_documentos_originais["sentencas"] = lista_documentos_originais["sentencas"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

lista_documentos_originais_pos["pos_documento"] = lista_documentos_originais_pos["pos_documento"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

logging.info("TERMINADO CORREÇÃO ORIGINAIS: {}.".format(len(lista_documentos_originais)))
logging.info("TERMINADO CORREÇÃO ORIGINAIS POS: {}.".format(len(lista_documentos_originais_pos)))

INFO:root:TERMINADO CORREÇÃO ORIGINAIS: 20.
INFO:root:TERMINADO CORREÇÃO ORIGINAIS POS: 20.


#### Criando dados indexados originais

In [ ]:
# Expecifica o(s) campo(s) indexado(s) e faz uma cópia da lista indexada
lista_documentos_originais_indexado = lista_documentos_originais.set_index(["id"])
lista_documentos_originais_indexado.head()

,sentencas,documento
id,,
1p0,[How to dequeue elements in a stack?],How to dequeue elements in a stack?
2p0,[How to dequeue elements in a stack?],How to dequeue elements in a stack?
3p0,[How to push elements in a queue?],How to push elements in a queue?
4p0,[How to push and pop elements in a queue?],How to push and pop elements in a queue?
5p0,[How to push elements in a queue data structure?],How to push elements in a queue data structure?


In [ ]:
# Expecifica o(s) campo(s) indexado(s) e faz uma cópia da lista indexada
lista_documentos_originais_pos_indexado = lista_documentos_originais_pos.set_index(["id"])
lista_documentos_originais_pos_indexado.head()

,pos_documento
id,
1p0,"[[[How, to, dequeue, elements, in, a, stack, ?..."
2p0,"[[[How, to, dequeue, elements, in, a, stack, ?..."
3p0,"[[[How, to, push, elements, in, a, queue, ?], ..."
4p0,"[[[How, to, push, and, pop, elements, in, a, q..."
5p0,"[[[How, to, push, elements, in, a, queue, data..."


#### Carrega o arquivo dos dados perturbados e POS

In [ ]:
# Abre o arquivo e retorna o DataFrame
lista_documentos_perturbados = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_PERTURBADO, sep=";", encoding="UTF-8")
lista_documentos_perturbados_pos = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_PERTURBADO_POS, sep=";", encoding="UTF-8")

logging.info("TERMINADO PERTURBADOS: {}.".format(len(lista_documentos_perturbados)))
logging.info("TERMINADO PERTURBADOS POS: {}.".format(len(lista_documentos_perturbados_pos)))

INFO:root:TERMINADO PERTURBADOS: 20.
INFO:root:TERMINADO PERTURBADOS POS: 20.


Alguns csv estão com o nome da coluna errado.

In [ ]:
lista_documentos_perturbados = lista_documentos_perturbados.rename(columns={'documentoPerturbado':'documento_perturbado'})

In [ ]:
lista_documentos_perturbados.sample(5)

,id,perturbado,documento_perturbado,sentencas
12,13p0_pert_0,['What is a stack and how to put an element in...,What is a stack and how to put an element in it ?,[['What is a stack and how to [MASK] an elemen...
6,7p0_pert_0,['How to remove elements from a queue ?'],How to remove elements from a queue ?,"[['How to [MASK] elements from a queue ?', 'po..."
18,19p0_pert_0,['In a stack does the enqueue operation specif...,In a stack does the enqueue operation specify ...,[['In a stack does the enqueue operation [MASK...
13,14p0_pert_0,['What is a queue and how to place an element ...,What is a queue and how to place an element on...,[['What is a queue and how to [MASK] an elemen...
11,12p0_pert_0,['What is a queue and how to extract an elemen...,What is a queue and how to extract an element ...,[['What is a queue and how to [MASK] an elemen...


In [ ]:
lista_documentos_perturbados.sample(5)

,id,perturbado,documento_perturbado,sentencas
13,14p0_pert_0,['What is a queue and how to place an element ...,What is a queue and how to place an element on...,[['What is a queue and how to [MASK] an elemen...
2,3p0_pert_0,['How to place elements in a queue ?'],How to place elements in a queue ?,"[['How to [MASK] elements in a queue ?', 'push..."
11,12p0_pert_0,['What is a queue and how to extract an elemen...,What is a queue and how to extract an element ...,[['What is a queue and how to [MASK] an elemen...
18,19p0_pert_0,['In a stack does the enqueue operation specif...,In a stack does the enqueue operation specify ...,[['In a stack does the enqueue operation [MASK...
9,10p0_pert_0,['What is a stack and how to define its elemen...,What is a stack and how to define its element ?,[['What is a stack and how to [MASK] its eleme...


In [ ]:
lista_documentos_perturbados_pos.sample(5)

,id,pos_documento
7,8p0_pert_0,"[[['How', 'to', 'manage', 'elements', 'in', 'a..."
16,17p0_pert_0,"[[['How', 'are', 'the', 'operations', 'to', 'p..."
10,11p0_pert_0,"[[['What', 'is', 'a', 'stack', 'and', 'how', '..."
6,7p0_pert_0,"[[['How', 'to', 'remove', 'elements', 'from', ..."
19,20p0_pert_0,"[[['In', 'a', 'queue', 'does', 'the', 'push', ..."


#### Corrigir os tipos de colunas dos dados perturbados e POS

Em dados perturbados:
- coluna 1 - `perturbado` carregadas do arquivo vem como string e não como lista.
- coluna 3 - `sentencas` carregadas do arquivo vem como string e não como lista.

Em dados perturbados pos:
- coluna 1 - `pos_documento` carregadas do arquivo vem como string e não como lista.

In [ ]:
# Import das bibliotecas.
import ast # Biblioteca para conversão de string em lista

# Verifica se o tipo da coluna não é list e converte
lista_documentos_perturbados["perturbado"] = lista_documentos_perturbados["perturbado"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)
lista_documentos_perturbados["sentencas"] = lista_documentos_perturbados["sentencas"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

lista_documentos_perturbados_pos["pos_documento"] = lista_documentos_perturbados_pos["pos_documento"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

logging.info("TERMINADO CORREÇÃO PERTURBADO: {}.".format(len(lista_documentos_perturbados)))
logging.info("TERMINADO CORREÇÃO PERTURBADO POS: {}.".format(len(lista_documentos_perturbados_pos)))

INFO:root:TERMINADO CORREÇÃO PERTURBADO: 20.
INFO:root:TERMINADO CORREÇÃO PERTURBADO POS: 20.


#### Criando dados indexados perturbados

In [ ]:
# Expecifica o(s) campo(s) indexado(s) e faz uma cópia da lista indexada
lista_documentos_perturbados_indexado = lista_documentos_perturbados.set_index(["id"])
lista_documentos_perturbados_indexado.head()

,perturbado,documento_perturbado,sentencas
id,,,
1p0_pert_0,[How to arrange elements in a stack ?],How to arrange elements in a stack ?,"[[How to [MASK] elements in a stack ?, dequeue..."
2p0_pert_0,[How to arrange elements in a stack ?],How to arrange elements in a stack ?,"[[How to [MASK] elements in a stack ?, dequeue..."
3p0_pert_0,[How to place elements in a queue ?],How to place elements in a queue ?,"[[How to [MASK] elements in a queue ?, push, p..."
4p0_pert_0,[How to rock and pop elements in a queue ?],How to rock and pop elements in a queue ?,"[[How to [MASK] and pop elements in a queue ?,..."
5p0_pert_0,[How to manage elements in a queue data struct...,How to manage elements in a queue data structu...,[[How to [MASK] elements in a queue data struc...


In [ ]:
# Expecifica o(s) campo(s) indexado(s) e faz uma cópia da lista indexada
lista_documentos_perturbados_pos_indexado = lista_documentos_perturbados_pos.set_index(["id"])
lista_documentos_perturbados_pos_indexado.head()

,pos_documento
id,
1p0_pert_0,"[[[How, to, arrange, elements, in, a, stack, ?..."
2p0_pert_0,"[[[How, to, arrange, elements, in, a, stack, ?..."
3p0_pert_0,"[[[How, to, place, elements, in, a, queue, ?],..."
4p0_pert_0,"[[[How, to, rock, and, pop, elements, in, a, q..."
5p0_pert_0,"[[[How, to, manage, elements, in, a, queue, da..."


### 5.1.5 Gerando pares de documentos originais e perturbados / Documento Original(1) e Documento Perturbado(0)


In [ ]:
# Import das bibliotecas.
import ast
from tqdm.notebook import tqdm as tqdm_notebook

print("Processando",len(lista_documentos_originais),"documentos originais")

lista_documentos_agrupados = []

# Barra de progresso dos documentos
lista_documentos_originais_bar = tqdm_notebook(lista_documentos_originais.iterrows(), desc=f"Documentos", unit=f" documento", total=len(lista_documentos_originais))

# Percorre os documentos
for i, linha_documento in lista_documentos_originais_bar:
  #if i < 2:
    #print("linha_documento:",linha_documento)
    # Recupera o id do documento
    id_documento_original = linha_documento[0]
    #print("id_documento_original:",id_documento_original)

    # Carrega a lista das sentenças do documento
    lista_sentencas_original = linha_documento[1]
    #print("\lista_sentencas_original:",lista_sentencas_original)
    #print("len(lista_sentencas_original):",len(lista_sentencas_original))

    # Carrega o documento original
    documento_original = linha_documento[2]
    #print("\documento_original:",documento_original)

    # Recupera a POS do documento original
    tokens_original = []
    tokens_original_pos = []
    reg_original_pos = lista_documentos_originais_pos_indexado.loc[id_documento_original]
    # print("reg_original_pos:",reg_original_pos)
    pos_documento_original = reg_original_pos['pos_documento']
    for i, linha2 in enumerate(pos_documento_original):

      tokens_original.append(linha2[0])
      tokens_original_pos.append(linha2[1])

    # Percorre os documentos perturbados apartir do original
    for j in range(0, model_args.documentos_perturbados):

        # Id do documento perturbado
        id_perturbado = str(id_documento_original) + "_pert_" + str(j)
        #print("id_perturbado:", id_perturbado)

        # Recupera o documento perturbado apartir do id original
        reg_documento_perturbado = lista_documentos_perturbados_indexado.loc[id_perturbado]
        # Recupera a sentença do documento perturbado
        lista_sentencas_perturbado = reg_documento_perturbado["perturbado"]
        #print("\lista_sentencas_perturbado:",lista_sentencas_perturbado)
        #print("len(lista_sentencas_perturbado):",len(lista_sentencas_perturbado))

        # Carrega o documento perturbado
        documento_perturbado = reg_documento_perturbado["documento_perturbado"]
        #print("\documento_perturbado:",documento_perturbado)

        # Recupera a POS do documento perturbado
        tokens_perturbado = []
        tokens_perturbado_pos = []
        reg_perturbado_pos = lista_documentos_perturbados_pos_indexado.loc[id_perturbado]
        #print("reg_perturbado_pos:",reg_perturbado_pos)
        pos_documento_perturbado = reg_perturbado_pos['pos_documento']
        for i, linha2 in enumerate(pos_documento_perturbado):
          tokens_perturbado.append(linha2[0])
          tokens_perturbado_pos.append(linha2[1])

        # Guarda o agrupamento de original e perturbado
        lista_documentos_agrupados.append([id_documento_original,
                                           lista_sentencas_original,
                                           documento_original,
                                           tokens_original,
                                           tokens_original_pos,
                                           id_perturbado,
                                           lista_sentencas_perturbado,
                                           documento_perturbado,
                                           tokens_perturbado,
                                           tokens_perturbado_pos])

logging.info("TERMINADO AGRUPAMENTO: {}.".format(len(lista_documentos_agrupados)))

Processando 20 documentos originais


Documentos:   0%|          | 0/20 [00:00<?, ? documento/s]

INFO:root:TERMINADO AGRUPAMENTO: 20.


#### Converte a lista em um dataframe

Atributos do dataframe:
Atributos do dataframe:
0. 'id_original' - Nome do arquivo original
1. 'sentencas_originais' - Lista das sentenças do documento original
2. 'documento_original' - Documento original
3. 'tokens_original' - Tokens do documento original
4. 'pos_original' - Postagging do documento original
5. 'id_perturbado' - Nome do arquivo perturbado
6. 'sentencas_perturbadas' - Lista das sentenças do documento perturbado
7. 'documento_perturbado' - Documento perturbado
8. 'tokens_perturbado' - Tokens do documento perturbado
9. 'pos_perturbado' - Postagging do documento perturbado

In [ ]:
# Import das bibliotecas.
import pandas as pd

# Converte a lista em um dataframe.
lista_documentos_agrupados = pd.DataFrame.from_records(lista_documentos_agrupados,
                                                         columns=['id_original',
                                                                  'sentencas_original',
                                                                  'documento_original',
                                                                  'tokens_original',
                                                                  'pos_original',
                                                                  'id_perturbado',
                                                                  'sentencas_perturbado',
                                                                  'documento_perturbado',
                                                                  'tokens_perturbado',
                                                                  'pos_perturbado'])

# Número de linhas carregadas do arquivo.
logging.info('Total de registros              : {}'.format(len(lista_documentos_agrupados)))

INFO:root:Total de registros              : 20


In [ ]:
lista_documentos_agrupados.sample(5)

,id_original,sentencas_original,documento_original,tokens_original,pos_original,id_perturbado,sentencas_perturbado,documento_perturbado,tokens_perturbado,pos_perturbado
1,2p0,[How to dequeue elements in a stack?],How to dequeue elements in a stack?,"[[How, to, dequeue, elements, in, a, stack, ?]]","[[SCONJ, PART, VERB, NOUN, ADP, DET, NOUN, PUN...",2p0_pert_0,[How to arrange elements in a stack ?],How to arrange elements in a stack ?,"[[How, to, arrange, elements, in, a, stack, ?]]","[[SCONJ, PART, VERB, NOUN, ADP, DET, NOUN, PUN..."
8,9p0,[What is a queue and how to push its element?],What is a queue and how to push its element?,"[[What, is, a, queue, and, how, to, push, its,...","[[PRON, AUX, DET, NOUN, CCONJ, SCONJ, PART, VE...",9p0_pert_0,[What is a queue and how to find its element ?],What is a queue and how to find its element ?,"[[What, is, a, queue, and, how, to, find, its,...","[[PRON, AUX, DET, NOUN, CCONJ, SCONJ, PART, VE..."
2,3p0,[How to push elements in a queue?],How to push elements in a queue?,"[[How, to, push, elements, in, a, queue, ?]]","[[SCONJ, PART, VERB, NOUN, ADP, DET, NOUN, PUN...",3p0_pert_0,[How to place elements in a queue ?],How to place elements in a queue ?,"[[How, to, place, elements, in, a, queue, ?]]","[[SCONJ, PART, VERB, NOUN, ADP, DET, NOUN, PUN..."
9,10p0,[What is a stack and how to enqueue its element?],What is a stack and how to enqueue its element?,"[[What, is, a, stack, and, how, to, enqueue, i...","[[PRON, AUX, DET, NOUN, CCONJ, SCONJ, PART, VE...",10p0_pert_0,[What is a stack and how to define its element ?],What is a stack and how to define its element ?,"[[What, is, a, stack, and, how, to, define, it...","[[PRON, AUX, DET, NOUN, CCONJ, SCONJ, PART, VE..."
14,15p0,[What is a queue and how to push and pop its e...,What is a queue and how to push and pop its el...,"[[What, is, a, queue, and, how, to, push, and,...","[[PRON, AUX, DET, NOUN, CCONJ, SCONJ, PART, VE...",15p0_pert_0,[What is a queue and how to push and pull its ...,What is a queue and how to push and pull its e...,"[[What, is, a, queue, and, how, to, push, and,...","[[PRON, AUX, DET, NOUN, CCONJ, SCONJ, PART, VE..."


Apaga as listas que não serão mais utilizadas

In [ ]:
del lista_documentos_originais
del lista_documentos_originais_pos
del lista_documentos_perturbados
del lista_documentos_perturbados_pos

## 5.2 Carrega os corpus específico

### Especifica os nomes dos arquivos do corpus

In [ ]:
# Nome do arquivo
NOME_ARQUIVO_CORPUS = "corpus_especifico.csv"
NOME_ARQUIVO_CORPUS_COMPACTADO = "corpus_especifico.zip"

### Copia os arquivos do Google Drive para o Colaboratory

In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_CORPUS_COMPACTADO" "$DIRETORIO_LOCAL"

  logging.info("Terminei a cópia.")

INFO:root:Terminei a cópia.


Descompacta os arquivos

Usa o unzip para descompactar:
*   `-o` sobrescreve o arquivo se existir
*   `-j` Não cria nenhum diretório
*   `-q` Desliga as mensagens
*   `-d` Diretório de destino


In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:
  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_CORPUS_COMPACTADO" -d "$DIRETORIO_LOCAL"

  logging.info("Terminei a descompactação.")

INFO:root:Terminei a descompactação.


### Carrega os dados

In [ ]:
# Import das bibliotecas.
import pandas as pd

# Abre o arquivo e retorna o DataFrame
df_corpus = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_CORPUS, sep=";", encoding="UTF-8")

print(len(df_corpus))

106


In [ ]:
df_corpus.sample(5)

,topico,fonte,sentenca
59,pilha,wikipedia,"Queues are common in computer programs, where ..."
41,pilha,Thomas Cormen,2 error \“underflow\”
1,pilha,wikipedia,For the use of the term pushdown in strength t...
35,pilha,Thomas Cormen,3 else return FALSE
18,pilha,Thomas Cormen,There are several efficient ways to implement ...


## 5.4 Medição

### 5.4.1 Wandb

https://wandb.ai/osmar-braz/MedidaCoerenciaCohebert_v1/table?workspace=user-osmar-braz

#### Função de inicialização wandb

In [ ]:
def inicializacaoWandb():

  if model_args.use_wandb:

    # Importando a biblioteca.
    import wandb

    #Login via linha de comando
    !wandb login aded3bc0ea651fff536cc08ba69caf8ac4141cfd

    # Inicializando o registro do experimento.
    # Na execução só pode existir de um init  para que não gere dois registros no wandb.
    wandb.init(project=NOME_BASE_SAIDA, name=NOME_BASE_SAIDA)

    # Atualiza os parâmetros do modelo no wandb.
    wandb.config.update(model_args)

    # Registra os parämetros não literais do model_args.
    wandb.log({'max_seq_len': model_args.max_seq_len})
    wandb.log({'do_lower_case': model_args.do_lower_case})
    wandb.log({'output_hidden_states': model_args.output_hidden_states})
    wandb.log({"documentos_perturbados": model_args.documentos_perturbados})
    wandb.log({"top_k_predicao": model_args.top_k_predicao})

    return wandb

### 5.4.2 Função quer realiza a medição de um documento



#### getMedidasTopica

In [ ]:
# Import das biblitecas
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel

def getCoerenciasTopica(textos, dados_palavras, tamanho_janela, numero_topicos = 2):

    # Criando o dicionário com as palavras a serem analisadas do corpus de referência e conjunto de dados
    id2word = corpora.Dictionary(textos)

    # Criando o corpus
    # Ocorrência das palavras a serem analisadas no corpus
    # Frequência de termos no documento (Term Document Frequency)
    corpus = [id2word.doc2bow(texto) for texto in textos]

    # Calcular pontuação de coerência 'c_uci'
    coherence_model = CoherenceModel(topics=dados_palavras,
                                     texts=textos,
                                     corpus=corpus,
                                     dictionary=id2word,
                                     window_size=tamanho_janela,
                                     coherence='c_uci')

    coherence_model_valor_u_uci = coherence_model.get_coherence()

    # Calcular pontuação de coerência 'c_npmi'
    coherence_model = CoherenceModel(topics=dados_palavras,
                                     texts=textos,
                                     dictionary=id2word,
                                     window_size=tamanho_janela,
                                     coherence='c_npmi')

    coherence_model_valor_c_npmi = coherence_model.get_coherence()

        # Calcular pontuação de coerência c_v
    coherence_model = CoherenceModel(topics=dados_palavras,
                                         texts=textos,
                                         corpus=corpus,
                                         dictionary=id2word,
                                         window_size=tamanho_janela,
                                         coherence='c_v')

    coherence_model_valor_c_v = coherence_model.get_coherence()

    # Calcular pontuação de coerência 'u_mass'
    coherence_model = CoherenceModel(topics=dados_palavras,
                                     texts=textos,
                                     corpus=corpus,
                                     dictionary=id2word,
                                     coherence='u_mass')

    coherence_model_valor_u_mass = coherence_model.get_coherence()


    return coherence_model_valor_u_uci, coherence_model_valor_c_npmi, coherence_model_valor_c_v, coherence_model_valor_u_mass

### 5.4.3 Função que realiza a medição de todos os documentos

In [ ]:
def dadosTratadoCorpus(texto, filtro_palavra):

  sentencia=False

  # Todas as palavras
  if filtro_palavra == 0:
    corpus = preparaCorpus(texto,
                           sentenciaTexto=sentencia,
                           tornaMinusculo=model_args.do_lower_case,
                           removePontuacao=False)
  else:
    # Sem as stopwords
    if filtro_palavra == 1:
      corpus = preparaCorpus(texto,
                             sentenciaTexto=sentencia,
                             tornaMinusculo=model_args.do_lower_case,
                             removePontuacao=False,
                             removeStopwords=True)
    else:
      # Sem as stopwords
      if filtro_palavra == 2:
        corpus = preparaCorpus(texto,
                               sentenciaTexto=sentencia,
                               tornaMinusculo=model_args.do_lower_case,
                               removePontuacao=False,
                               removeStopwords=False,
                               somenteRelevante=True)
      else:
        # Todas as palavras e sem pontuação
        if filtro_palavra == 3:
          corpus = preparaCorpus(texto,
                                sentenciaTexto=sentencia,
                                tornaMinusculo=model_args.do_lower_case,
                                removePontuacao=True)
        else:
          # Sem as stopwords e sem pontuação
          if filtro_palavra == 4:
            corpus = preparaCorpus(texto,
                                  sentenciaTexto=sentencia,
                                  tornaMinusculo=model_args.do_lower_case,
                                  removePontuacao=True,
                                  removeStopwords=True)
          else:
            # Sem as stopwords e sem pontuação
            if filtro_palavra == 5:
              corpus = preparaCorpus(texto,
                                    sentenciaTexto=sentencia,
                                    tornaMinusculo=model_args.do_lower_case,
                                    removePontuacao=True,
                                     removeStopwords=False,
                                    somenteRelevante=True)

  return corpus

In [ ]:
# Import das bibliotecas.
from tqdm.notebook import tqdm as tqdm_notebook

def calculaMedidasDocumentos(corpus,
                             lista_documentos_agrupados,
                             wandb,
                             tamanho_janela,
                             filtro_palavra):

  '''
    Percorre os documentos para calcular as medidas das sentenças
  '''

  logging.info("Processando {} pares de documentos originais e perturbados.".format(len(lista_documentos_agrupados)))

  # Contadores de ocorrência de coerência
  conta_coherence_model_valor_u_uci = 0
  conta_coherence_model_valor_c_npmi = 0
  conta_coherence_model_valor_c_v = 0
  conta_coherence_model_valor_c_umass = 0

  conta = 0

  # Retorna os dados tratados do corpus
  corpus_tratado = dadosTratadoCorpus(corpus['sentenca'].values.tolist(),filtro_palavra)
  # print("corpus_tratado:",corpus_tratado)

  # Lista para o salvamento das medidas
  lista_medidas_documentos_salvar = []

  # Barra de progresso dos documentos
  lista_documentos_agrupados_bar = tqdm_notebook(lista_documentos_agrupados.iterrows(), desc=f"Documentos", unit=f" documento", total=len(lista_documentos_agrupados))

  # Percorre os documentos do conjunto de dados
  for i, linha_documento in lista_documentos_agrupados_bar:
    # if i < 5:

      # Conta o número de pares de documentos
      conta = conta + 1

      #print("linha_documento:",linha_documento)
      # Recupera o id do documento Original
      id_documento_original = linha_documento[0]
      # print("id_documento_original:",id_documento_original)
      lista_sentenca_documento_original = linha_documento[1]
      #print("lista_sentenca_documento_original:",lista_sentenca_documento_original)
      #print("len(lista_sentenca_documento_original):",len(lista_sentenca_documento_original))
      # Recupera o documento Original
      documento_original = linha_documento[2]
      # print("documento_original:",documento_original)
      # Recupera os tokens do documento original
      lista_tokens_documento_original = linha_documento[3]
      #print("lista_tokens_documento_original:",lista_tokens_documento_original)
      #print("len(lista_tokens_documento_original):",len(lista_tokens_documento_original))
      # Recupera o postagging do documento original
      lista_pos_documento_original = linha_documento[4]
      #print("lista_pos_documento_original:",lista_pos_documento_original)
      #print("len(lista_pos_documento_original):",len(lista_pos_documento_original))

      # Calcula a coerencia do documento
      # Retorna os dados tratados do documento
      doc_original_tratado = dadosTratadoCorpus([documento_original],filtro_palavra)
      # print("doc_original_tratado:",doc_original_tratado)
      # print("corpus_tratado:",corpus_tratado)

      coherence_model_valor_u_uci_orig, coherence_model_valor_c_npmi_orig, coherence_model_valor_c_v_orig, coherence_model_valor_c_umass_orig = getCoerenciasTopica(corpus_tratado, doc_original_tratado, tamanho_janela)

      # Recupera o id do documento Perturbado
      id_documento_perturbado = linha_documento[5]
      # print("id_documento_perturbado:",id_documento_perturbado)
      lista_sentenca_documento_perturbado = linha_documento[6]
      #print("lista_sentenca_documento_perturbado:",lista_sentenca_documento_perturbado)
      #print("len(lista_sentenca_documento_perturbado):",len(lista_sentenca_documento_perturbado))
      # Recupera o documento Perturbado
      documento_perturbado = linha_documento[7]
      #print("documento_perturbado:",documento_perturbado)
      # Recupera os tokens do documento perturbado
      lista_tokens_documento_perturbado = linha_documento[8]
      #print("lista_tokens_documento_perturbado:",lista_tokens_documento_perturbado)
      #print("len(lista_tokens_documento_perturbado):",len(lista_tokens_documento_perturbado))
      # Recupera o postagging do documento original
      lista_pos_documento_perturbado = linha_documento[9]
      #print("lista_pos_documento_perturbado:",lista_pos_documento_perturbado)
      #print("len(lista_pos_documento_perturbado):",len(lista_pos_documento_perturbado))

      # Calcula a coerencia do documento
      # Retorna os dados tratados do documento
      doc_perturbado_tratado = dadosTratadoCorpus([documento_perturbado],filtro_palavra)

      # print("doc_perturbado_tratado:",doc_perturbado_tratado)
      coherence_model_valor_u_uci_pert, coherence_model_valor_c_npmi_pert, coherence_model_valor_c_v_pert, coherence_model_valor_c_umass_pert = getCoerenciasTopica(corpus_tratado,doc_perturbado_tratado, tamanho_janela)

      # Verifica a medida de coerência u_uci das sentenças do documento original com as sentenças do documento pertubado.
      # Quanto menor o valor de Ceuc mais as documentos do documentos são coerentes
      if coherence_model_valor_u_uci_orig >= coherence_model_valor_u_uci_pert:
          conta_coherence_model_valor_u_uci = conta_coherence_model_valor_u_uci + 1

      # Verifica a medida de coerência c_npmi das sentenças do documento original com as sentenças do documento pertubado.
      # Quanto menor o valor de Ceuc mais as documentos do documentos são coerentes
      if coherence_model_valor_c_npmi_orig >= coherence_model_valor_c_npmi_pert:
          conta_coherence_model_valor_c_npmi = conta_coherence_model_valor_c_npmi + 1

      # Verifica a medida de coerência c_v das sentenças do documento original com as sentenças do documento pertubado.
      # Quanto maior o valor de coherence_model_valor_c_v_orig mais as documentos do documentos são coerentes
      if coherence_model_valor_c_v_orig >= coherence_model_valor_c_v_pert:
          conta_coherence_model_valor_c_v = conta_coherence_model_valor_c_v + 1

      # Verifica a medida de coerência c_umass das sentenças do documento original com as sentenças do documento pertubado.
      # Quanto menor o valor de Ceuc mais as documentos do documentos são coerentes
      if coherence_model_valor_c_umass_orig >= coherence_model_valor_c_umass_pert:
          conta_coherence_model_valor_c_umass = conta_coherence_model_valor_c_umass + 1

      # Guarda as medidas em uma lista para salvar em arquivo
      # Guarda as medidas dos documentos originais
      lista_medidas_documentos_salvar.append([id_documento_original,
                                              coherence_model_valor_u_uci_orig,
                                              coherence_model_valor_c_npmi_orig,
                                              coherence_model_valor_c_v_orig,
                                              coherence_model_valor_c_umass_orig
                                              ])
      # Guarda as medidas dos documentos perturbados
      lista_medidas_documentos_salvar.append([id_documento_perturbado,
                                              coherence_model_valor_u_uci_pert,
                                              coherence_model_valor_c_npmi_pert,
                                              coherence_model_valor_c_v_pert,
                                              coherence_model_valor_c_umass_pert])

  logging.info("Total de Pares : {}.".format(str(conta)))

  if model_args.use_wandb:
       wandb.log({'pares_doc': conta})

  logging.info("Pares Corretos u_uci {}.".format(str(conta_coherence_model_valor_u_uci)))
  acuracia_u_uci = float(conta_coherence_model_valor_u_uci)/float(conta)
  logging.info("Acurácia: {}.".format(str(acuracia_u_uci*100)))

  if model_args.use_wandb:
    wandb.log({'acuracia_u_uci': acuracia_u_uci})

  logging.info("Pares Corretos c_npmi {}.".format(str(conta_coherence_model_valor_c_npmi)))
  acuracia_c_npmi = float(conta_coherence_model_valor_c_npmi)/float(conta)
  logging.info("Acurácia: {}.".format(str(acuracia_c_npmi*100)))

  if model_args.use_wandb:
    wandb.log({'acuracia_c_npmi': acuracia_c_npmi})

  logging.info("Pares Corretos c_v: {}.".format(str(conta_coherence_model_valor_c_v)))
  acuracia_c_v = float(conta_coherence_model_valor_c_v)/float(conta)
  logging.info("Acurácia: {}.".format(str(acuracia_c_v*100)))

  if model_args.use_wandb:
    wandb.log({'acuracia_c_v': acuracia_c_v})

  logging.info("Pares Corretos u_mass {}.".format(str(conta_coherence_model_valor_c_umass)))
  acuracia_c_umass = float(conta_coherence_model_valor_c_umass)/float(conta)
  logging.info("Acurácia: {}.".format(str(acuracia_c_umass*100)))

  if model_args.use_wandb:
    wandb.log({'acuracia_c_umass': acuracia_c_umass})

  logging.info("TERMINADO!")

  del lista_documentos_agrupados_bar

  return lista_medidas_documentos_salvar, conta, acuracia_u_uci, conta_coherence_model_valor_u_uci, acuracia_c_npmi, conta_coherence_model_valor_c_npmi, acuracia_c_v, conta_coherence_model_valor_c_v, acuracia_c_umass, conta_coherence_model_valor_c_umass

### 5.4.4 Salvando os resultados

#### Salvando o resultado da medição

In [ ]:
def salvaResultadoMedicao(lista_medidas_documentos_salvar):

  if model_args.salvar_medicao:

    # Import das bibliotecas.
    import os
    import datetime

    # Recupera a hora do sistema.
    data_e_hora = datetime.datetime.now()

    FILTRO_PALAVRA = '_tap' # Todas as palavras
    if model_args.filtro_palavra == 1:
      FILTRO_PALAVRA = '_ssw'  # Sem stopwords
    else:
      if model_args.filtro_palavra == 2:
        FILTRO_PALAVRA = '_svs'  # Somente verbos(e auxiliares) e substantivos
      else:
        if model_args.filtro_palavra == 3:
          FILTRO_PALAVRA = '_sp_tap'  # Todas as palavras e sem pontuação
        else:
          if model_args.filtro_palavra == 4:
            FILTRO_PALAVRA = '_sp_ssw'  # Sem stopwords e substantivos e sem pontuação
          else:
            if model_args.filtro_palavra == 5:
              FILTRO_PALAVRA = '_sp_svs'  # Somente verbos(e auxiliares) e substantivos e sem pontuação

    # Contatena os parâmetros que forma o nome do arquivo medição
    NOME_ARQUIVO_MEDICAO = NOME_BASE_SAIDA + "_P_" + str(model_args.documentos_perturbados) + "_K_" + str(model_args.top_k_predicao) + "_TJ_" + str(model_args.tamanho_janela) + FILTRO_PALAVRA

    # Diretório do drive do arquivo
    DIRETORIO_MEDICAO_DRIVE = DIRETORIO_DRIVE + "validacao_medicao_topico_palavra/Medicao/"

    # Diretório local para salvar o arquivo
    DIRETORIO_MEDICAO_LOCAL = DIRETORIO_LOCAL + "Medicao/"

    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_MEDICAO_DRIVE):
      # Cria o diretório
      os.makedirs(DIRETORIO_MEDICAO_DRIVE)
      logging.info("Diretório criado: {}.".format(DIRETORIO_MEDICAO_DRIVE))
    else:
      logging.info("Diretório já existe: {}.".format(DIRETORIO_MEDICAO_DRIVE))

    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_MEDICAO_LOCAL):
      # Cria o diretório
      os.makedirs(DIRETORIO_MEDICAO_LOCAL)
      logging.info("Diretório criado: {}.".format(DIRETORIO_MEDICAO_LOCAL))
    else:
      logging.info("Diretório já existe: {}.".format(DIRETORIO_MEDICAO_LOCAL))

    # Caminho completo do arquivo compactado no drive
    NOME_ARQUIVO_MEDICAO_DRIVE_COMPACTADO = DIRETORIO_MEDICAO_DRIVE + NOME_ARQUIVO_MEDICAO + ".zip"
    # print("NOME_ARQUIVO_MEDICAO_DRIVE_COMPACTADO:", NOME_ARQUIVO_MEDICAO_DRIVE_COMPACTADO)

    # Caminho completo do arquivo compactado no local
    NOME_ARQUIVO_MEDICAO_LOCAL_COMPACTADO = DIRETORIO_MEDICAO_LOCAL + NOME_ARQUIVO_MEDICAO + ".zip"
    # print("NOME_ARQUIVO_MEDICAO_LOCAL_COMPACTADO:", NOME_ARQUIVO_MEDICAO_LOCAL_COMPACTADO)

    # Caminho completo do arquivo no local
    NOME_ARQUIVO_MEDICAO_LOCAL = DIRETORIO_MEDICAO_LOCAL + NOME_ARQUIVO_MEDICAO + ".csv"
    # print("NOME_ARQUIVO_MEDICAO_LOCAL:", NOME_ARQUIVO_MEDICAO_LOCAL)

    # Gera todo o conteúdo a ser salvo no arquivo
    novo_conteudo = ''
    for resultado in lista_medidas_documentos_salvar:
      novo_conteudo = novo_conteudo + data_e_hora.strftime('%d/%m/%Y %H:%M') + ';'  + str(resultado[0]) + ';'  + str(resultado[1]) + ';'  + str(resultado[2]) + ';'  + str(resultado[3]) + ';' + str(resultado[4]) + '\n'

    # Verifica se o arquivo existe.
    if os.path.isfile(NOME_ARQUIVO_MEDICAO_DRIVE_COMPACTADO):
      # Copia arquivo da medição compactado do google drive para o drive local
      !cp "$NOME_ARQUIVO_MEDICAO_DRIVE_COMPACTADO" "$NOME_ARQUIVO_MEDICAO_LOCAL_COMPACTADO"
      # Descompacta arquivo da medição compactado no drive local
      !unzip -o -j -q "$NOME_ARQUIVO_MEDICAO_LOCAL_COMPACTADO" -d "$DIRETORIO_MEDICAO_LOCAL"

      logging.info("Atualizando arquivo medição: {}.".format(NOME_ARQUIVO_MEDICAO_LOCAL))
      # Abre o arquivo para leitura.
      arquivo = open(NOME_ARQUIVO_MEDICAO_LOCAL,'r')
      # Leitura de todas as linhas do arquivo.
      conteudo = arquivo.readlines()
      # Conteúdo a ser adicionado.
      conteudo.append(novo_conteudo)

      # Abre novamente o arquivo (escrita).
      arquivo = open(NOME_ARQUIVO_MEDICAO_LOCAL,'w')
      # Escreve o conteúdo criado anteriormente nele.
      arquivo.writelines(conteudo)
      # Fecha o arquivo.
      arquivo.close()

      # Compacta o arquivo da medição
      !zip -o -q -j "$NOME_ARQUIVO_MEDICAO_LOCAL_COMPACTADO" "$NOME_ARQUIVO_MEDICAO_LOCAL"
      # Copia o arquivo da medição compactado para o drive
      !cp "$NOME_ARQUIVO_MEDICAO_LOCAL_COMPACTADO" "$NOME_ARQUIVO_MEDICAO_DRIVE_COMPACTADO"

    else:
      logging.info("Criando arquivo medição: {}.".format(NOME_ARQUIVO_MEDICAO_LOCAL))
      # Abre novamente o arquivo (escrita).
      arquivo = open(NOME_ARQUIVO_MEDICAO_LOCAL,'w')
      arquivo.writelines('data;arquivo;c_uci;c_npmi;c_v;c_umass\n' + novo_conteudo)  # escreva o conteúdo criado anteriormente nele.
      # Fecha o arquivo.
      arquivo.close()

      # Compacta o arquivo da medição
      !zip -o -q -j "$NOME_ARQUIVO_MEDICAO_LOCAL_COMPACTADO" "$NOME_ARQUIVO_MEDICAO_LOCAL"
      # Copia o arquivo da medição compactado para o drive
      !cp "$NOME_ARQUIVO_MEDICAO_LOCAL_COMPACTADO" "$NOME_ARQUIVO_MEDICAO_DRIVE_COMPACTADO"

#### Salvando o resultado da avaliação

In [ ]:
def salvaResultadoAvaliacao(tempo_total_processamento,
                            conta,
                            acuracia_u_uci,
                            conta_coherence_model_valor_u_uci,
                            acuracia_c_npmi,
                            conta_coherence_model_valor_c_npmi,
                            acuracia_c_v,
                            conta_coherence_model_valor_c_v,
                            acuracia_c_umass,
                            conta_coherence_model_valor_c_umass):

  if model_args.salvar_avaliacao:

    # Import das bibliotecas.
    import os

    # Recupera a hora do sistema.
    data_e_hora = datetime.datetime.now()

    FILTRO_PALAVRA = '_tap' # Todas as palavras
    if model_args.filtro_palavra == 1:
      FILTRO_PALAVRA = '_ssw'  # Sem stopwords
    else:
      if model_args.filtro_palavra == 2:
        FILTRO_PALAVRA = '_svs'  # Somente verbos(e auxiliares) e substantivos
      else:
        if model_args.filtro_palavra == 3:
          FILTRO_PALAVRA = '_sp_tap'  # Todas as palavras e sem pontuação
        else:
          if model_args.filtro_palavra == 4:
            FILTRO_PALAVRA = '_sp_ssw'  # Sem stopwords e substantivos e sem pontuação
          else:
            if model_args.filtro_palavra == 5:
              FILTRO_PALAVRA = '_sp_svs'  # Somente verbos(e auxiliares) e substantivos e sem pontuação

    # Nome arquivo resultado
    NOME_ARQUIVO_AVALIACAO = NOME_BASE_SAIDA + "_P_" + str(model_args.documentos_perturbados) + "_K_" + str(model_args.top_k_predicao) + "_TJ_" + str(model_args.tamanho_janela) + FILTRO_PALAVRA

    # Diretório para salvar o arquivo de resultado.
    DIRETORIO_AVALIACAO = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_COHEBERT + "/validacao_medicao_topico_palavra/Avaliacao/"

    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_AVALIACAO):
      # Cria o diretório
      os.makedirs(DIRETORIO_AVALIACAO)
      logging.info("Diretório criado: {}.".format(DIRETORIO_AVALIACAO))
    else:
      logging.info("Diretório já existe: {}.".format(DIRETORIO_AVALIACAO))

    # Nome do arquivo a ser aberto.
    NOME_ARQUIVO_AVALIACAO_COMPLETO = DIRETORIO_AVALIACAO + NOME_ARQUIVO_AVALIACAO + '.csv'

    # Conteúdo a ser adicionado.
    novo_conteudo = (NOME_ARQUIVO_AVALIACAO + ';' +
                     data_e_hora.strftime('%d/%m/%Y %H:%M') + ';' +
                     tempo_total_processamento + ';' +
                     str(conta) + ';' +
                     str(acuracia_u_uci) + ';' +
                     str(conta_coherence_model_valor_u_uci) + ';' +
                     str(acuracia_c_npmi) + ';' +
                     str(conta_coherence_model_valor_c_npmi) + ";" +
                     str(acuracia_c_v) + ';' +
                     str(conta_coherence_model_valor_c_v) + ';' +
                     str(acuracia_c_umass) + ';' +
                     str(conta_coherence_model_valor_c_umass) + '\n')

    # Verifica se o arquivo existe.
    if os.path.isfile(NOME_ARQUIVO_AVALIACAO_COMPLETO):
      logging.info("Atualizando arquivo resultado avaliação: {}.".format(NOME_ARQUIVO_AVALIACAO_COMPLETO))
      # Abre o arquivo para leitura.
      arquivo = open(NOME_ARQUIVO_AVALIACAO_COMPLETO,'r')
      # Leitura de todas as linhas do arquivo.
      conteudo = arquivo.readlines()
      # Conteúdo a ser adicionado.
      conteudo.append(novo_conteudo)

      # Abre novamente o arquivo (escrita).
      arquivo = open(NOME_ARQUIVO_AVALIACAO_COMPLETO,'w')
      # escreva o conteúdo criado anteriormente nele.
      arquivo.writelines(conteudo)
      # Fecha o arquivo.
      arquivo.close()
    else:
      logging.info("Criando arquivo resultado avaliação: {}.".format(NOME_ARQUIVO_AVALIACAO_COMPLETO))
      # Abre novamente o arquivo (escrita).
      arquivo = open(NOME_ARQUIVO_AVALIACAO_COMPLETO,'w')
      arquivo.writelines('arquivo;data;tempo;conta;c_uci;contac_uci;c_npmi;contac_npmi;c_v;contac_v;c_umass;contac_umass\n' + novo_conteudo)  # escreva o conteúdo criado anteriormente nele.
      # Fecha o arquivo.
      arquivo.close()

### 5.4.5 Função de cálculo das medidas dos documentos

In [ ]:
# Import das bibliotecas.
import time
import datetime
import gc

def procedimentoCalculaMedida(tamanho_janela, filtro_palavra):

  # Seta o parâmetro do tamanho da janela
  model_args.tamanho_janela = tamanho_janela

  # Seta o parâmetro do fitro (ALL,CLEAN,VERNOUN)
  model_args.filtro_palavra = filtro_palavra

  logging.info("Processamento tamanho janela {} e filtro palavra {}.".format(tamanho_janela, FILTRO_PALAVRAS_STR[filtro_palavra]))

  # Marca o tempo de início do processamento
  tempoInicioTeste = time.time()
  logging.info("Tempo início processamento: {:} (h:mm:ss).".format(formataTempo(tempoInicioTeste)))

  # Inicializa o wandb para registro
  wandb = inicializacaoWandb()

  # Calcula as medidas dos documentos
  resultado_medida, conta, acuracia_u_uci, conta_coherence_model_valor_u_uci, acuracia_c_npmi, conta_coherence_model_valor_c_npmi, acuracia_c_v, conta_coherence_model_valor_c_v, acuracia_c_umass, conta_coherence_model_valor_c_umass = calculaMedidasDocumentos(df_corpus, lista_documentos_agrupados, wandb, tamanho_janela, filtro_palavra)

  # Pega o tempo atual menos o tempo do início do processamento.
  tempoFinalTeste = time.time()
  tempo_total_processamento = formataTempo(tempoFinalTeste - tempoInicioTeste)

  # Salva o resultado da classificação
  salvaResultadoMedicao(resultado_medida)

  # Salva o resultado da avaliação
  salvaResultadoAvaliacao(tempo_total_processamento,
                          conta,
                          acuracia_u_uci,
                          conta_coherence_model_valor_u_uci,
                          acuracia_c_npmi,
                          conta_coherence_model_valor_c_npmi,
                          acuracia_c_v,
                          conta_coherence_model_valor_c_v,
                          acuracia_c_umass,
                          conta_coherence_model_valor_c_umass)

  logging.info("  Tempo processamento: {:} (h:mm:ss).\n".format(tempo_total_processamento))

  # Finaliza o wandb
  if model_args.use_wandb:
     wandb.finish()

  # Apaga as variáveis
  del resultado_medida
  del wandb

  # Chama o coletor de lixo para esvaziar a memória
  gc.collect()

### 5.4.6 Executa o procedimento para todos os parâmetros

In [ ]:
# Import das bibliotecas.
from tqdm.notebook import tqdm as tqdm_notebook

# Barra de progresso do tamanho da janela
tamanho_janela_bar = tqdm_notebook(range(TAMANHO_JANELA_INICIO,TAMANHO_JANELA_FIM+1), desc=f'Tamanho da janela', unit=f'filtro', total=TAMANHO_JANELA_FIM-TAMANHO_JANELA_INICIO)

# Percorre todos os tamanhos de janela
for tamanho_janela in tamanho_janela_bar:

  # Barra de progresso filtro (0 - ALL, 1 - CLEAN, 2 - VERBNOUN)
  filtro_palavra_bar = tqdm_notebook(enumerate(FILTRO_PALAVRAS), desc=f'Filtro palavras', unit=f'filtro', total=len(FILTRO_PALAVRAS))

  # Percorre todos formas de filtro de palavras a serem avaliados
  for filtro_palavra_i, filtro_palavra in filtro_palavra_bar:

    # Passa os parâmetros para o procedimento cálculo das medidas
    procedimentoCalculaMedida(tamanho_janela, filtro_palavra)

Tamanho da janela:   0%|          | 0/8 [00:00<?, ?filtro/s]

Filtro palavras:   0%|          | 0/3 [00:00<?, ?filtro/s]

INFO:root:Processamento tamanho janela 142 e filtro palavra SEM_PONTUACAO_TODAS_AS_PALAVRAS.
INFO:root:Tempo início processamento: 19312 days, 10:46:01 (h:mm:ss).


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: osmar-braz. Use `wandb login --relogin` to force relogin


...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


INFO:root:Processando 20 pares de documentos originais e perturbados.


Documentos:   0%|          | 0/20 [00:00<?, ? documento/s]

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary<0 unique tokens: []>
INFO:gensim.corpora.dictionary:built Dictionary<289 unique tokens: ['accounting', 'lifo', 'see', 'term', 'use']...> from 104 documents (total 805 corpus positions)
INFO:gensim.utils:Dictionary lifecycle event {'msg': "built Dictionary<289 unique tokens: ['accounting', 'lifo', 'see', 'term', 'use']...> from 104 documents (total 805 corpus positions)", 'datetime': '2022-11-16T10:55:01.193364', 'gensim': '4.2.0', 'python': '3.7.15 (default, Oct 12 2022, 19:14:55) \n[GCC 7.5.0]', 'platform': 'Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}
INFO:gensim.topic_coherence.probability_estimation:using WordOccurrenceAccumulator to estimate probabilities from sliding windows
INFO:gensim.topic_coherence.probability_estimation:using WordOccurrenceAccumulator to estimate probabilities from sliding windows
INFO:gensim.topic_coherence.probability_estimation:using WordOccurrenceAccumulator to estim

acuracia_c_npmi,▁
acuracia_c_umass,▁
acuracia_c_v,▁
acuracia_u_uci,▁
do_lower_case,▁
documentos_perturbados,▁
output_hidden_states,▁
pares_doc,▁
top_k_predicao,▁
acuracia_c_npmi,0.2
acuracia_c_umass,0.1


# 6 Finalização

## 6.1 Tempo final de processamento



In [ ]:
 # Pega o tempo atual menos o tempo do início do processamento.
final_processamento = time.time()
tempo_total_processamento = formataTempo(final_processamento - inicio_processamento)

print('')
print('  Tempo processamento:  {:} (h:mm:ss)'.format(tempo_total_processamento))


  Tempo processamento:  0:09:57 (h:mm:ss)
